In [2]:
import os
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
VER = 1

In [3]:
BASE_PATH = "/Users/choeyeonghwan1/Desktop/HMS/hms-harmful-brain-activity-classification/"

df_egg = pd.read_parquet(BASE_PATH + "train_eegs/1000913311.parquet")

df_egg.head()

,Fp1,F3,C3,P3,F7,T3,T5,O1,Fz,Cz,Pz,Fp2,F4,C4,P4,F8,T4,T6,O2,EKG
0,-105.849998,-89.230003,-79.459999,-49.230000,-99.730003,-87.769997,-53.330002,-50.740002,-32.250000,-42.099998,-43.270000,-88.730003,-74.410004,-92.459999,-58.930000,-75.739998,-59.470001,8.210000,66.489998,1404.930054
1,-85.470001,-75.070000,-60.259998,-38.919998,-73.080002,-87.510002,-39.680000,-35.630001,-76.839996,-62.740002,-43.040001,-68.629997,-61.689999,-69.320000,-35.790001,-58.900002,-41.660000,196.190002,230.669998,3402.669922
2,8.840000,34.849998,56.430000,67.970001,48.099998,25.350000,80.250000,48.060001,6.720000,37.880001,61.000000,16.580000,55.060001,45.020000,70.529999,47.820000,72.029999,-67.180000,-171.309998,-3565.800049
3,-56.320000,-37.279999,-28.100000,-2.820000,-43.430000,-35.049999,3.910000,-12.660000,8.650000,3.830000,4.180000,-51.900002,-21.889999,-41.330002,-11.580000,-27.040001,-11.730000,-91.000000,-81.190002,-1280.930054
4,-110.139999,-104.519997,-96.879997,-70.250000,-111.660004,-114.430000,-71.830002,-61.919998,-76.150002,-79.779999,-67.480003,-99.029999,-93.610001,-104.410004,-70.070000,-89.250000,-77.260002,155.729996,264.850006,4325.370117


In [4]:
# each column is a channel

n_channels = df_egg.shape[1]
n_channels

20

In [5]:
df = pd.read_csv("/Users/choeyeonghwan1/Desktop/HMS/hms-harmful-brain-activity-classification/train.csv")

TARGETS = df.columns[-6:]
print('Train shape:', df.shape)
print('Targets',list(TARGETS))

df.head()

Train shape: (106800, 15)
Targets ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']


,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


In [7]:
# CREATE NON-OVERLAPPING EEG ID TRAIN DATA

train = df.groupby('eeg_id')[['spectrogram_id',
                            'spectrogram_label_offset_seconds']].agg({
    'spectrogram_id':'first','spectrogram_label_offset_seconds':'min'
}) ## ??

train.head()



,spectrogram_id,spectrogram_label_offset_seconds
eeg_id,,
568657,789577333,0.0
582999,1552638400,0.0
642382,14960202,1008.0
751790,618728447,908.0
778705,52296320,0.0


In [11]:
train.columns = ['spec_id','min']

train.head()

,spec_id,min
eeg_id,,
568657,789577333,0.0
582999,1552638400,0.0
642382,14960202,1008.0
751790,618728447,908.0
778705,52296320,0.0


In [16]:
tmp = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
                            {'spectrogram_label_offset_seconds':'max'})

train['max'] = tmp
train.head()

,spec_id,min,max
eeg_id,,,
568657,789577333,0.0,16.0
582999,1552638400,0.0,38.0
642382,14960202,1008.0,1032.0
751790,618728447,908.0,908.0
778705,52296320,0.0,0.0


In [19]:
tmp = df.groupby('eeg_id')[['patient_id']].agg('first')
train['patient_id'] = tmp

train.head()

,spec_id,min,max,patient_id
eeg_id,,,,
568657,789577333,0.0,16.0,20654
582999,1552638400,0.0,38.0,20230
642382,14960202,1008.0,1032.0,5955
751790,618728447,908.0,908.0,38549
778705,52296320,0.0,0.0,40955


In [20]:
tmp = df.groupby('eeg_id')[TARGETS].agg('sum')

for t in TARGETS:
    train[t] = tmp[t].value

train.head()

AttributeError: 'Series' object has no attribute 'value'